In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import theano
import matplotlib.pyplot as plt
import arviz as az
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as st
#from more_itertools import repeat_each
from patsy import dmatrix, dmatrices

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### 12.2 Data generation

In [2]:
np.random.seed(1234)
n_groups = 56
n_sample = 10
n = n_groups * n_sample
#pop = list(repeat_each(range(1, n_groups + 1), n_sample)
pop = np.repeat(range(1, n_groups + 1), n_sample)

# Body length
original_length = st.uniform.rvs(45, 25, size = n)
mn = original_length.mean()
sd = original_length.std(ddof = 1)
#print("Mean and sd used to normalise.original length:", mn, sd, "\n")
length = (original_length - mn) / sd

data_ds = pd.DataFrame({'pop': pd.Series(pop, dtype = 'category'), 'length': length})
Xmat = dmatrix("~ pop * length - 1 - length", data_ds, return_type = 'dataframe')

intercept_mean = 230      # mu_alpha
intercept_sd = 20         # sigma_alpha
slope_mean = 60           # mu_beta
slope_sd = 30             # sigma_beta

intercept_effects = st.norm.rvs(intercept_mean, intercept_sd, size = n_groups)
slope_effects = st.norm.rvs(slope_mean, slope_sd, size = n_groups)
all_effects = np.concatenate((intercept_effects, slope_effects))

lin_pred = Xmat.to_numpy() @ all_effects
eps = st.norm.rvs(0, 30, size = n)
mass = lin_pred + eps
data = pd.DataFrame({'mass': mass, 
                     'pop': pd.Series(pop, dtype = 'category'), 
                     'length': length})

In [3]:
#fig, ax = plt.subplots(2,1,figsize=(8,12))
#ax[0].hist(length, bins = 8, range = [-2, 2], color= 'gray', edgecolor='black', alpha = 0.5)
#ax[1].hist(mass, bins = 10, range = [0, 500], color= 'gray', edgecolor='black', alpha = 0.5);

**FIGURE 12.2**

In [4]:
fig, axes = plt.subplots(7, 8, figsize=(20, 18), sharey=True, sharex=True, dpi=500)
fig.subplots_adjust(left=0.075, right=0.975, bottom=0.09, top=0.925, wspace=0.03)

axes_flat = axes.ravel()

for i, pop_ in enumerate(range(1, n_groups + 1)):
    ax = axes_flat[i]
    idx_ = data.index[data["pop"] == pop_].tolist()
    mass_ = data.loc[idx_, "mass"].values
    length_ = data.loc[idx_, "length"].values

    # Plot observed data points
    ax.scatter(length_, mass_, color="white", ec="royalblue", alpha=0.7)

    # Add a title
    ax.set_title(f"Pop: {pop_}", fontsize=12)

fig.text(0.5, 0.02, "Length", fontsize=14)
fig.text(0.03, 0.5, "Mass", rotation=90, fontsize=14, va="center");

### 12.3 Analysis under a random-intercepts model

#### 12.3.2 Bayesian analysis using PyMC3

In [5]:
idx = pd.factorize(pop)[0]
groups = len(np.unique(idx))

In [6]:
with pm.Model() as model_12_3_2:
    mu_int = pm.Normal('mu_int', 0, tau = 0.001)                      # Mean hyperparameter for random intercepts
    sigma_int = pm.Uniform('sigma_int', 0, 100)                       # SD hyperparameter for random intercepts
    alpha = pm.Normal('alpha', mu_int, sigma_int, shape = groups)     # Random intercepts
    
    beta = pm.Normal('beta', 0, tau = 0.001)                          # Common slope
    sigma = pm.Uniform('sigma', 0, 100)                               # Residual standard deviation
    
    mu = pm.Deterministic('mu', alpha[idx] + beta * length)
    mass_new = pm.Normal('mass_new', mu = mu, sigma = sigma, observed = mass)
    
    trace_12_3_2 = pm.sample(return_inferencedata = True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha, sigma_int, mu_int]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 27 seconds.


In [7]:
az.summary(trace_12_3_2, var_names = ['beta', 'mu_int', 'sigma_int', 'sigma'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta,55.372,1.817,51.884,58.772,0.029,0.020,4004.0,1309.0,1.0
mu_int,221.104,3.532,214.454,227.416,0.073,0.051,2328.0,1377.0,1.0
sigma_int,22.597,2.977,16.911,27.964,0.069,0.049,1719.0,1088.0,1.0
sigma,41.557,1.363,39.030,44.073,0.024,0.017,3102.0,1351.0,1.0


### 12.4 Random-coefficients model without correlation between intercept and slope

#### 12.4.2 Bayesian analysis using PyMC3

In [8]:
with pm.Model() as model_12_4_2:
    mu_int = pm.Normal('mu_int', 0, tau = 0.001)                          # Mean hyperparameter for random intercepts
    sigma_int = pm.Uniform('sigma_int', 0, 100)                           # SD hyperparameter for random intercepts
    alpha = pm.Normal('alpha', mu_int, sigma_int, shape = groups)         # Random intercepts
    
    mu_slope = pm.Normal('mu_slope', 0, tau = 0.001)                      # Mean hyperparameter for random slopes
    sigma_slope = pm.Uniform('sigma_slope', 0, 100)                       # SD hyperparameter for slopes
    beta = pm.Normal('beta', mu_slope, sigma_slope, shape = groups)       # Random slopes
    
    sigma = pm.Uniform('sigma', 0, 100)                                   # Residual standard deviation
    
    mu = pm.Deterministic('mu', alpha[idx] + beta[idx] * length)
    mass_new = pm.Normal('mass_new', mu = mu, sigma = sigma, observed = mass)
    
    trace_12_4_2 = pm.sample(return_inferencedata = True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, sigma_slope, mu_slope, alpha, sigma_int, mu_int]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 46 seconds.


In [9]:
az.summary(trace_12_4_2, var_names = ['mu_int', 'sigma_int', 'mu_slope', 'sigma_slope', 'sigma'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_int,223.645,3.113,217.331,228.944,0.072,0.051,1865.0,1461.0,1.0
sigma_int,20.638,2.460,15.913,24.937,0.069,0.049,1233.0,1514.0,1.0
mu_slope,54.904,4.051,47.386,62.476,0.086,0.061,2252.0,1467.0,1.0
sigma_slope,29.283,3.252,23.307,35.068,0.082,0.059,1612.0,1472.0,1.0
sigma,30.313,1.005,28.447,32.202,0.025,0.017,1661.0,1431.0,1.0


### 12.5. The random-coefficients model with correlation between intercept and slope

#### 12.5.2 Data Generation

In [10]:
np.random.seed(1234)
n_groups = 56
n_sample = 10
n = n_groups * n_sample 
pop = np.repeat(np.arange(1, n_groups+1), n_sample)

original_length = np.random.uniform(45, 70, n)                 # Body length (cm)
mn = np.mean(original_length)
sd = np.std(original_length, ddof = 1)
#print("Mean and sd used to normalise.original length:", mn, sd, "\n")
length = (original_length - mn) / sd

pop_mat = pd.get_dummies(pop).to_numpy()
length_mat = np.zeros((n, n_groups))
for i, j in zip(range(0, n-n_sample+1, n_sample), range(n_groups)):
    length_mat[i:i+n_sample, j] = length[i:i+n_sample]

X_mat = np.concatenate((pop_mat, length_mat), axis = 1)

intercept_mean = 230                                           # Values for five hyperparameters
intercept_sd = 20
slope_mean = 60
slope_sd = 30
intercept_slope_covariance = 10
mu_vector = np.array([intercept_mean, slope_mean])
var_cova_matrix = np.array([[intercept_sd**2,intercept_slope_covariance], 
                            [intercept_slope_covariance, slope_sd**2]])

effects = np.random.multivariate_normal(mean=mu_vector, cov=var_cova_matrix, size=n_groups)
#print(np.mean(effects, axis = 0))
#print(np.cov(effects.T))
all_effects = np.ravel(effects, order = 'F')

lin_pred = X_mat @ all_effects
eps = st.norm.rvs(0, 30, size = n)
mass = lin_pred + eps

In [11]:
idx = pd.factorize(pop)[0]
groups = len(np.unique(idx))

In [12]:
with pm.Model() as model_15_5_3:
    
    chol, Rho_, sigma_pop = pm.LKJCholeskyCov("chol_cov", n=2, eta=2, sd_dist=pm.Uniform.dist(0,100), compute_corr=True)

    alpha = pm.Normal("alpha", 0, tau = 0.001)      # prior for average intercept
    beta = pm.Normal("beta", 0, tau = 0.001)        # prior for average slope
    ab_pop = pm.MvNormal("ab_pop", mu=tt.stack([alpha, beta]), chol=chol, shape=(groups, 2)) 
    mu = ab_pop[idx, 0] + ab_pop[idx, 1] * length     # linear model
    sigma_within = pm.Uniform("sigma_within", 0,100)  # prior stddev within populations

    mass_pred = pm.Normal("mass_pred", mu=mu, sigma=sigma_within, observed=mass)  # likelihood

    trace_15_5_3 = pm.sample(4000, target_accept=0.9, return_inferencedata = True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_within, ab_pop, beta, alpha, chol_cov]


Sampling 2 chains for 1_000 tune and 4_000 draw iterations (2_000 + 8_000 draws total) took 171 seconds.


In [13]:
az.summary(trace_15_5_3, var_names=["~ab_pop", "~chol_cov"], round_to=2)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,224.98,2.93,219.45,230.49,0.03,0.02,10485.49,6486.12,1.0
beta,53.69,4.09,46.20,61.61,0.03,0.02,13971.85,6894.91,1.0
chol_cov_stds[0],19.59,2.50,14.98,24.23,0.03,0.02,8633.27,6455.62,1.0
chol_cov_stds[1],28.72,3.16,22.79,34.64,0.03,0.02,10513.62,6347.49,1.0
"chol_cov_corr[0, 0]",1.00,0.00,1.00,1.00,0.00,0.00,8000.00,8000.00,NaN
"chol_cov_corr[0, 1]",0.08,0.15,-0.22,0.35,0.00,0.00,9675.15,6905.43,1.0
"chol_cov_corr[1, 0]",0.08,0.15,-0.22,0.35,0.00,0.00,9675.15,6905.43,1.0
"chol_cov_corr[1, 1]",1.00,0.00,1.00,1.00,0.00,0.00,7894.44,7603.95,1.0
sigma_within,30.32,1.03,28.34,32.17,0.01,0.01,10332.82,6547.10,1.0
